In [132]:
import pickle
from PIL import Image
import numpy as np
from tqdm import tqdm
from konlpy.tag import Komoran

In [133]:
target_size = 256
image_data, sentiment_data, sentence_data = pickle.load(open('data.pkl', 'rb'))

In [134]:
print(image_data.shape)
print(sentiment_data.shape)
print(sentence_data.shape)

(7000, 256, 256, 3)
(7000, 4)
(7000,)


In [135]:
komoran = Komoran()
output_tags = [komoran.pos(text, join=True) for text in sentence_data]
output_tags = [[tag.split('/')[0] for tag in tags] for tags in output_tags]

valid_idx = []
temp = []
for i, tags in enumerate(output_tags):
    tags.insert(0, '\t')
    tags.append('\n')
    if len(tags) <= 60:
        valid_idx.append(i)
        temp.append(tags)
        
image_data = image_data[valid_idx]
sentiment_data = sentiment_data[valid_idx]
sentence_data = sentence_data[valid_idx]
output_tags = temp

In [136]:
from collections import OrderedDict

len_dict = {}
for i in map(len, output_tags):
    if i in len_dict:
        len_dict[i] += 1
    else:
        len_dict[i] = 1
len_dict = OrderedDict(sorted(len_dict.items()))
len_dict

OrderedDict([(3, 7),
             (4, 63),
             (5, 157),
             (6, 205),
             (7, 204),
             (8, 197),
             (9, 210),
             (10, 209),
             (11, 167),
             (12, 191),
             (13, 176),
             (14, 149),
             (15, 151),
             (16, 144),
             (17, 131),
             (18, 138),
             (19, 95),
             (20, 101),
             (21, 97),
             (22, 93),
             (23, 96),
             (24, 99),
             (25, 100),
             (26, 87),
             (27, 76),
             (28, 76),
             (29, 79),
             (30, 88),
             (31, 87),
             (32, 81),
             (33, 78),
             (34, 67),
             (35, 52),
             (36, 55),
             (37, 64),
             (38, 61),
             (39, 76),
             (40, 72),
             (41, 56),
             (42, 70),
             (43, 67),
             (44, 65),
             (45, 50),
   

In [156]:
# RNN 준비과정

output_words = set()
for tags in output_tags:
    for word in tags:
        if word not in output_words:
            output_words.add(word)
            
output_words = sorted(list(output_words))
output_words[0], output_words[1] = output_words[1], output_words[0]
num_decoder_tokens = len(output_words)
max_decoder_seq_length = max(map(len, output_tags))

output_token_index = {w: i for i, w in enumerate(output_words)}
reverse_output_char_index = dict((i, w) for w, i in output_token_index.items())

# decoder_output_data = np.zeros(shape=(7000, max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_output_data = np.zeros(shape=(sentence_data.shape[0], max_decoder_seq_length), dtype='float32')

for i, s in enumerate(output_tags):
    for j, c in enumerate(s):
        if j > 0:
            decoder_output_data[i, j-1] = output_token_index[c]

In [138]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

tf.reset_default_graph()

In [139]:
# 이미지 (CNN)

X = tf.placeholder(tf.float32, shape=[None, target_size, target_size, 3])

n_conv_1 = 64
k_conv_1 = 5
W_conv_1 = tf.Variable(tf.truncated_normal(shape=[k_conv_1,k_conv_1,3,n_conv_1], stddev=5e-2))
b_conv_1 = tf.Variable(tf.constant(0.1, shape=[n_conv_1]))
h_conv_1 = tf.nn.relu(tf.nn.conv2d(X, W_conv_1, strides=[1,1,1,1], padding='SAME') + b_conv_1)
h_pool_1 = tf.nn.max_pool(h_conv_1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

n_conv_2 = 64
k_conv_2 = 5
W_conv_2 = tf.Variable(tf.truncated_normal(shape=[k_conv_2,k_conv_2,n_conv_1,n_conv_2], stddev=5e-2))
b_conv_2 = tf.Variable(tf.constant(0.1, shape=[n_conv_2]))
h_conv_2 = tf.nn.relu(tf.nn.conv2d(h_pool_1, W_conv_2, strides=[1,1,1,1], padding='SAME') + b_conv_2)
h_pool_2 = tf.nn.max_pool(h_conv_2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

n_conv_3 = 128
k_conv_3 = 3
W_conv_3 = tf.Variable(tf.truncated_normal(shape=[k_conv_3,k_conv_3,n_conv_2,n_conv_3], stddev=5e-2))
b_conv_3 = tf.Variable(tf.constant(0.1, shape=[n_conv_3]))
h_conv_3 = tf.nn.relu(tf.nn.conv2d(h_pool_2, W_conv_3, strides=[1,1,1,1], padding='SAME') + b_conv_3)
h_pool_3 = tf.nn.max_pool(h_conv_3, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

n_conv_4 = 128
k_conv_4 = 3
W_conv_4 = tf.Variable(tf.truncated_normal(shape=[k_conv_4,k_conv_4,n_conv_3,n_conv_4], stddev=5e-2))
b_conv_4 = tf.Variable(tf.constant(0.1, shape=[n_conv_4]))
h_conv_4 = tf.nn.relu(tf.nn.conv2d(h_pool_3, W_conv_4, strides=[1,1,1,1], padding='SAME') + b_conv_4)

n_conv_5 = 128
k_conv_5 = 3
W_conv_5 = tf.Variable(tf.truncated_normal(shape=[k_conv_5,k_conv_5,n_conv_4,n_conv_5], stddev=5e-2))
b_conv_5 = tf.Variable(tf.constant(0.1, shape=[n_conv_5]))
h_conv_5 = tf.nn.relu(tf.nn.conv2d(h_conv_4, W_conv_5, strides=[1,1,1,1], padding='SAME') + b_conv_5)
h_conv_5_flat = tf.reshape(h_conv_5, [-1, h_conv_5.shape[1]*h_conv_5.shape[2]*h_conv_5.shape[3]])

n_fc_1 = max_decoder_seq_length * 5
k_conv_1 = 5
W_fc_1 = tf.Variable(tf.truncated_normal(shape=[h_conv_5_flat.shape[1], n_fc_1], stddev=5e-2))
b_fc_1 = tf.Variable(tf.constant(0.1, shape=[n_fc_1]))
h_fc_1 = tf.nn.relu(tf.matmul(h_conv_5_flat, W_fc_1) + b_fc_1)

h_fc_1_drop = tf.nn.dropout(h_fc_1, 0.8)

In [140]:
# 감정

# n_senti = 4

# S = tf.placeholder(tf.float32, shape=[None, n_senti])

# h_pre_s = tf.tile(h_fc_1_drop, [1, n_senti]) * tf.repeat(S, n_fc_1, axis=1)

# n_final = 256
# W_final = tf.Variable(tf.truncated_normal(shape=[n_senti*n_fc_1, n_final]))
# b_final = tf.Variable(tf.constant(0.1, shape=[n_final]))
# h_final = tf.matmul(h_pre_s, W_final) + b_final

In [141]:
# 문장 생성 (RNN)

learning_rate = 1e-2
n_hidden = 1024
n_class = n_input = num_decoder_tokens

# enc_input = tf.reshape(h_final, [-1, n_final, 1])
enc_input = tf.reshape(h_fc_1_drop, [-1, max_decoder_seq_length, 5])
dec_output = tf.placeholder(tf.int64, [None, max_decoder_seq_length])

dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.8)
outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, enc_input, dtype=tf.float32,
                                        scope='decoder')

model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=dec_output))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [142]:
model, dec_output

(<tf.Tensor 'dense/BiasAdd:0' shape=(?, 60, 9682) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?, 60) dtype=int64>)

In [202]:
from collections import OrderedDict

len_dict = {}
for s in decoder_output_data:
    for i in s:
        if i in len_dict:
            len_dict[i] += 1
        else:
            len_dict[i] = 1
len_dict = OrderedDict(sorted(len_dict.items()))
len_dict2 = {}
count = 0
for i, n in len_dict.items():
    if n in len_dict2:
        len_dict2[n] += 1
    else:
        len_dict2[n] = 1
len_dict2 = OrderedDict(sorted(len_dict2.items()))
len_dict2

OrderedDict([(1, 4724),
             (2, 1433),
             (3, 714),
             (4, 407),
             (5, 328),
             (6, 241),
             (7, 197),
             (8, 138),
             (9, 119),
             (10, 110),
             (11, 92),
             (12, 79),
             (13, 68),
             (14, 49),
             (15, 54),
             (16, 55),
             (17, 35),
             (18, 27),
             (19, 30),
             (20, 37),
             (21, 39),
             (22, 35),
             (23, 15),
             (24, 25),
             (25, 24),
             (26, 31),
             (27, 13),
             (28, 17),
             (29, 12),
             (30, 13),
             (31, 19),
             (32, 12),
             (33, 19),
             (34, 9),
             (35, 6),
             (36, 13),
             (37, 12),
             (38, 7),
             (39, 7),
             (40, 6),
             (41, 14),
             (42, 11),
             (43, 14),
             

In [197]:
output_words[51]

'ㅂ니다요'

In [144]:
tf.trainable_variables()

[<tf.Variable 'Variable:0' shape=(5, 5, 3, 64) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'Variable_2:0' shape=(5, 5, 64, 64) dtype=float32_ref>,
 <tf.Variable 'Variable_3:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'Variable_4:0' shape=(3, 3, 64, 128) dtype=float32_ref>,
 <tf.Variable 'Variable_5:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'Variable_6:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 <tf.Variable 'Variable_7:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'Variable_8:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 <tf.Variable 'Variable_9:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'Variable_10:0' shape=(131072, 300) dtype=float32_ref>,
 <tf.Variable 'Variable_11:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'decoder/basic_rnn_cell/kernel:0' shape=(1029, 1024) dtype=float32_ref>,
 <tf.Variable 'decoder/basic_rnn_cell/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(10

In [145]:
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                            log_device_placement=True))
    sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5



In [146]:
W_conv_1.shape

TensorShape([Dimension(5), Dimension(5), Dimension(3), Dimension(64)])

In [154]:
total_epoch = 100
train_size = 1000
valid_size = 200
batch_size = 100
n_batch = train_size // batch_size
n_valid_batch = valid_size // batch_size

valid_image = image_data[train_size:train_size+valid_size]
valid_sentiments = sentiment_data[train_size:train_size+valid_size]
valid_input_data = np.zeros(shape=(valid_size, max_decoder_seq_length))
valid_output_data = np.zeros(shape=(valid_size, max_decoder_seq_length))

for i, s in enumerate(output_tags[train_size:train_size+valid_size]):
    for j, c in enumerate(s):
        if j > 0:
            valid_output_data[i, j-1] = output_token_index[c]
            
dec_output_data = np.zeros(shape=(train_size, max_decoder_seq_length))

for i, s in enumerate(output_tags[:train_size]):
    for j, c in enumerate(s):
        if j > 0:
            dec_output_data[i, j-1] = output_token_index[c]

for epoch in range(total_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
        batch_images = image_data[batch_size*batch:batch_size*(batch+1)]
        batch_sentiments = sentiment_data[batch_size*batch:batch_size*(batch+1)]
        batch_dec_outputs = dec_output_data[batch_size*batch:batch_size*(batch+1)]

        _, loss = sess.run([optimizer, cost], feed_dict={
            X: batch_images,
            dec_output: batch_dec_outputs
        })

        total_loss += loss
        
    total_valid_loss = 0
    for batch in range(n_valid_batch):
        valid_inp = [output_token_index['\t']] + [output_token_index['\n']]*(max_decoder_seq_length-1)
        valid_inp = np.array([np.eye(num_decoder_tokens)[valid_inp]] * n_batch)
        valid_loss = sess.run(cost, feed_dict={
            X: valid_image[batch_size*batch:batch_size*(batch+1)],
            dec_output: valid_output_data[batch_size*batch:batch_size*(batch+1)]
        })
        total_valid_loss += valid_loss
        

    print('Epoch: %04d Cost: %.6f Valid Cost: %.6f' %(epoch + 1, total_loss / n_batch, valid_loss))

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0001 Cost: 3.852855 Valid Cost: 4.574734


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0002 Cost: 3.750632 Valid Cost: 4.551154


  0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch: 0041 Cost: 3.814705 Valid Cost: 4.707369


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0042 Cost: 3.807227 Valid Cost: 4.663939


100%|██████████| 10/10 [00:03<00:00,  3.05it/s]


KeyboardInterrupt: 

In [153]:
valid_output_data[batch_size*batch:batch_size*(batch+1)]

array([], shape=(0, 60), dtype=float64)

In [150]:
batch_dec_outputs.shape

(100, 60)

In [174]:
def translate(image, senti):
    prediction = tf.argmax(model, 2)
    # dec_out = [output_token_index['\t']*(max_decoder_seq_length-1) + output_token_index['']]
    # dec_out = np.eye(dict_len)[dec_out]
    result = sess.run(prediction, 
                      feed_dict={X: image})
    decoded = [output_words[i] for i in result[0]]
    end = decoded.index('\n') if '\n' in decoded else len(decoded)
    translated = ''.join(decoded[:end])
    return translated

In [175]:
for idx in range(10):
    print(sentence_data[idx:idx+1])
    print(translate(image_data[idx:idx+1], sentiment_data[idx:idx+1]))

['\t나쁜에너지를 내뿜는 사람들을 끊어내라 많은 사람을 곁에 두려 하지 말고 긍정적인 사람들을 곁에 두려 노력하라 좋은사람에게만 좋은사람이면돼에서\n']
오늘
['\t따뜻한가요\n']
오늘
['\t맛집 오늘의 포인트 이없다는거\n']
오늘
['\t기억 안에선 가장 맑은 하늘\n']
오늘
['\t오늘 하늘 너무 이쁘다\n']
오늘
['\t우오오오오옹 전역 일차 날씨 짱좋다아아아아\n']
날씨
['\t하늘이너무이쁘다\n']
오늘
['\t하늘이 맑고 푸르다\n']
오늘
['\t얼마만에 이런 하늘이야 넘 좋다\n']
오늘ㄴ
['\t오랜만에 맑은 하늘\n']
오늘


In [81]:
prediction = tf.argmax(model, 2)
sess.run(prediction, feed_dict={X: image_data[:10]})

array([[1045,    2,    2, ...,    0,    0,    0],
       [1045,    2,    2, ...,    0,    0,    0],
       [1045,    2,    2, ...,    0,    0,    0],
       ...,
       [1045,    2,    2, ...,    0,    0,    0],
       [1045,    2,    2, ...,    0,    0,    0],
       [1045,    2,    2, ...,    0,    0,    0]])